# Time Series Analysis Notes

My notes on DataCamp's [Time Series Analysis in Python](https://campus.datacamp.com/courses/introduction-to-time-series-analysis-in-python/) course (1-4) and my notes on exponential smoothing with code ported from R (5):

### Contents
1. [Understanding the Dickey-Fuller Test](#dftest)
2. [Auto-Regressive (AR) Models](#ar)
3. [Moving Average (MA) and ARMA Models](#ma)
4. [Cointegration Models](#coint)

<a id='dftest'></a>
### Understanding the Dickey-Fuller Test

In a random walk, the current value is equal to the last value plus some noise:

\begin{equation}
y_t = ay_{t-1} + \epsilon_t
\end{equation}

If the value of $a$ is 1, then each term is equal to the last, except for the added error. The random walk is "integrated of order 1," in that we must difference it once to get a stationary series. (A stationary series does not change its mean or variance over time.)

If the difference between two points in time is due completely to the added error, there should be no contributing slope component in the difference between two values:

\begin{equation}
y_t - y_{t-1} = (\phi - 1)y_{t-1} + \epsilon_t
\end{equation}

That is, mathematically, if there's a unit root is present in the current value formula ($a=1$), when we check the correlation between the difference series and the lagged series, the slope (correlation) coefficient of the lagged series ($\phi - 1$) should be zero.

This is what the Dickey-Fuller Test investigates: the Dickey-Fuller Test correlates a series' difference against its lag. If the correlation coefficient (slope) is 0 or very close to 0, the difference is just the noise and the current value series is a random walk.

<a id='ar'></a>
### Auto-Regressive (AR) Models

In an AR model of order 1 (AR(1)), today's value is a mean plus a fraction of yesterday's value plus noise:

\begin{equation}
y_t = \mu + \phi y_{t-1} + \epsilon_t
\end{equation}

An AR(1) series can be stationary if $-1 \lt \phi \lt 1$. A positive value for $\phi$ means the series will behave a bit like a random walk and exhibit momentum, while a negative value means that values will alternate sign and exhibit mean reversion (and the series looks a bit like a waveform).

Intuitively: if $\phi$ is negative, the current value is the flipped sign of the last value, and the path of the series alternates its direction every other value. A positive $\phi$ doesn't result in this alternation.

The autocorrelation function of an AR(1) series decays exponentially: the lag(1) series will have autocorrelation $\phi$, lag(2) $\phi^2$, lag(3) $\phi^3$, and lag(n) $\phi^n$. A negative value just reverses the sign of the autocorrelation at each lag: the comparison alternates between same and opposite signs (see intuition above). Describing the current value with two values back is AR(2), three back AR(3), n back AR(n).

In [ ]:
# Simulating an AR process: general example
from statsmodels.tsa.arima_process import ArmaProcess
ar = np.array([1, -0.9]) # zero-lag coefficient of 1, phi value of 0.9 (! it's the opposite because signal processing)
ma = np.array([1])
AR_object = ArmaProcess(ar, ma)
simulated_data = AR_object.generate_sample(nsample=1000)
plt.plot(simulated_data)

In [ ]:
# Simulating an AR process: specific example with plot
# import the module for simulating data
from statsmodels.tsa.arima_process import ArmaProcess

# Plot 1: AR parameter = +0.9
plt.subplot(2,1,1)
ar1 = np.array([1, -0.9])
ma1 = np.array([1])
AR_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = AR_object1.generate_sample(nsample=1000)
plt.plot(simulated_data_1)

# Plot 2: AR parameter = -0.9
plt.subplot(2,1,2)
ar2 = np.array([1, 0.9])
ma2 = np.array([1])
AR_object2 = ArmaProcess(ar2, ma2)
simulated_data_2 = AR_object2.generate_sample(nsample=1000)
plt.plot(simulated_data_2)
plt.show()

#### Estimating and Forecasting an AR Model

Here is how to estimate the some data set's $\phi$, assuming an AR(1) model:

In [ ]:
# Import the ARMA module from statsmodels
from statsmodels.tsa.arima_model import ARMA

# Fit an AR(1) model to the first simulated data
mod = ARMA(simulated_data_1, order=(1,0)) # fit to AR(1) model
res = mod.fit()

# Print out summary information on the fit
print(res.summary())

# Print out the estimate for the constant and for phi
print("When the true phi=0.9, the estimate of phi (and the constant) are:")
print(res.params)

And here's how to forecast a particular date range with an AR(1) model using the `predict()` and `plot_predict()` methods. "In-sample" forecasting guesses one value after the existing data, while "out-of-sample" forecasts some values into the future.

In [ ]:
# Import the ARMA module from statsmodels
from statsmodels.tsa.arima_model import ARMA

# Forecast the first AR(1) model
mod = ARMA(simulated_data_1, order=(1,0))
res = mod.fit()
res.plot_predict(start=990, end=1010) # point 990 to point 1010 out of a 1,000 value series
plt.show()

Sometimes it's tough to tell the difference between a random walk and a time series that mean reverts a little bit.

#### Choosing the Right Model

How do you know the best order $p$ for an AR($p$) model? The partial autocorrelation function (PACF) and the information criteria help you choose the order of your model.

The PACF measures the incremental benefit of adding another lag to the model function. `plot_pacf()` works like `plot_acf()`: one series argument and two kwargs, lag and alpha. The number of lags significantly (out of the blue signficance band you choose with alpha) different than zero tells you the order of the model.

Information Criteria prevents overfitting by assessing a penalty for more model parameters. Two goodness-of-fit measures:

1. AIC( Akaike Information Criterion) `result.aic`
2. BIC (Bayesian Information Criterion) `result.bic`

In practice, fit several models and then check out the BIC for each. Choose the model with the lowest value for the best fit.

Here's how to plot the PACF for a series:

In [ ]:
# Import the modules for simulating data and for plotting the PACF
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_pacf

# Simulate AR(1) with phi=+0.6
ma = np.array([1])
ar = np.array([1, -0.6])
AR_object = ArmaProcess(ar, ma)
simulated_data_1 = AR_object.generate_sample(nsample=5000)

# Plot PACF for AR(1)
plot_pacf(simulated_data_1, lags=20)
plt.show()

# Simulate AR(2) with phi1=+0.6, phi2=+0.3
ma = np.array([1])
ar = np.array([1, -0.6, -0.3])
AR_object = ArmaProcess(ar, ma)
simulated_data_2 = AR_object.generate_sample(nsample=5000)

# Plot PACF for AR(2)
plot_pacf(simulated_data_2, lags=20)
plt.show()

Here's how to plot the BIC as a function of model order:

In [ ]:
# Import the module for estimating an ARMA model
from statsmodels.tsa.arima_model import ARMA

# Fit the data to an AR(p) for p = 0,...,6 , and save the BIC
BIC = np.zeros(7)
for p in range(7):
    mod = ARMA(simulated_data_2, order=(p,0))
    res = mod.fit()
# Save BIC for AR(p)    
    BIC[p] = res.bic
    
# Plot the BIC as a function of p
plt.plot(range(1,7), BIC[1:7], marker='o')
plt.xlabel('Order of AR Model')
plt.ylabel('Bayesian Information Criterion')
plt.show()

<a id='ma'></a>
### Moving Average (MA) and ARMA Models

In an MA(1) model (moving average model of order 1), today's value is a mean pluse noise plus a fraction theta of yesterday's noise:

\begin{equation}
y_t = \mu + \epsilon + \theta \epsilon_{t-1}
\end{equation}

This is of order 1, because there's one lagged value in the equation.

If $\theta$ is 0, the process is white noise shifted by a mean. MA models are stationary for all values of theta.

If \theta is negative, then a positive value last period means the next value is likely to be negative; a positive value two values might not have any effect two values later.

lag(1) autocorrelation for MA models is not $\theta$ but $\theta/(1+\theta)^2$. When $\theta$ is negative, lag(1) autocorrelation is negative. When $\theta$ is positive, lag(1) autocorrelation is positive. An MA(n) has no autocorrelation beyond lag(n).

Here's how to simulate an MA process:

In [ ]:
# import the module for simulating data
from statsmodels.tsa.arima_process import ArmaProcess

# Plot 1: MA parameter = -0.9
plt.subplot(2,1,1)
ar1 = np.array([1])
ma1 = np.array([1, -.9])
MA_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = MA_object1.generate_sample(nsample=1000)
plt.plot(simulated_data_1)

# Plot 2: MA parameter = +0.9
plt.subplot(2,1,2)
ar2 = np.array([1])
ma2 = np.array([1, 0.9])
MA_object2 = ArmaProcess(ar2, ma2)
simulated_data_2 = MA_object2.generate_sample(nsample=1000)
plt.plot(simulated_data_2)

plt.show()

Here's how to compute the autocorrelation for an MA series (as before):

In [ ]:
# Import the plot_acf module from statsmodels
from statsmodels.graphics.tsaplots import plot_acf

# Plot 1: MA parameter = -0.9
plot_acf(simulated_data_1, lags=20)
plt.show()

#### Estimating and Forecasting an MA Model

Just use statsmodel's `ARMA` class as with an AR model, but pass in `order=(0,1)` in place of `order=(1,0)` for an MA model. You can forecast a single value after data, and then one more value after that, after which all values will be the same.

#### ARMA Models

In an AR model, the current value has to do with n previous values. In an MA model, the current value has to do with n previous noise components. In an ARMA model, the current value has to do with both past values and past noise components.

In an ARMA(1, 1) model, the current value is equal to a mean plus some fraction $\phi$ of the last value plus some noise component plus some fraction $\theta$ of the last noise component.

\begin{equation}
y_t = \mu + \phi y_{t-1} + \epsilon + \theta \epsilon_{t-1}
\end{equation}

It's possible to convert an ARMA  model of order n into an AR model of order inifinity or an MA model of order infinity by substitution.

*Neat Trick:* It' useful to find the indices of missing data by subtracting the index from a complete range:

In [ ]:
# Everything
set_everything = set(range(391))

# The intraday index as a set
set_intraday = set(intraday.index)

# Calculate the difference
set_missing = set_everything - set_intraday

# Print the difference
print("Missing rows: ", set_missing)

# reindex with forward fill to replace missing values
intraday = intraday.reindex(range(391), method='ffill')

#### Transforming a numeric index into a datetime index

Pandas has a `date_range` method. Make a date range and set it as the index:

In [ ]:
# From previous step
intraday = intraday.reindex(range(391), method='ffill')

# Change the index to the intraday times
intraday.index = pd.date_range(start='2017-09-01 9:30', end='2017-09-01 16:00', freq='1min')

# Plot the intraday time series
intraday.plot(grid=True)
plt.show()

#### Applying an MA Model

If you see no autocorrelations at lags higher than n, you should fit an MA(n) model to the data. Compute the difference series using `pct_change()` and then fit an MA model to the difference series, remembering to use `drop_na()` to get rid of the first row's `NaN` (because differences).

In [ ]:
# Import plot_acf and ARMA modules from statsmodels
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima_model import ARMA

# Compute returns from prices and drop the NaN
returns = intraday.pct_change()
returns = returns.dropna()

# Plot ACF of returns with lags up to 60 minutes
plot_acf(returns, lags=60)
plt.show()

# Fit the data to an MA(1) model
mod = ARMA(returns, order=(0,1))
res = mod.fit()
print(res.params)

#### Simulating an AR(1) model as an MR($\infty$) Model

An AR(1) model is the same as an MR($\infty$) model:

\begin{equation}
y_t = \mu + \phi y_{t-1} + \epsilon
\end{equation}

\begin{equation}
y_t = \mu + \sum_{i=1}^{\infty} \phi^i \epsilon_{t-i}
\end{equation}

Here's a simulation of an AR(1) model with $\phi$=0.8 as an MR($\infty$) model:

In [ ]:
# import the modules for simulating data and plotting the ACF
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf

# Build a list MA parameters
ma = [0.8**i for i in range(30)]

# Simulate the MA(30) model
ar = np.array([1])
AR_object = ArmaProcess(ar, ma)
simulated_data = AR_object.generate_sample(nsample=5000)

# Plot the ACF
plot_acf(simulated_data, lags=30)
plt.show()

<a id='coint'></a>
### Cointegration Models